In [44]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
# application_df.head()
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [45]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df = application_df.drop(["EIN","NAME"],1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [46]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
application_df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [47]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
apptype_count=application_df.APPLICATION_TYPE.value_counts()
apptype_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [48]:
# Choose a cutoff value and create a list of application types to be replaced
app_types_to_replace = list(apptype_count[apptype_count < 500].index)

# Replace in dataframe
for app in app_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [49]:
# Look at CLASSIFICATION value counts for binning
classification_cnt=application_df.CLASSIFICATION.value_counts()
print(classification_cnt)
print("-"*50)
print(f"Average: {classification_cnt.mean()}")
print(f"Median: {classification_cnt.median()}") 

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64
--------------------------------------------------
Average: 483.0845070422535
Median: 5.0


In [50]:
# You may find it helpful to look at CLASSIFICATION value counts >1

cls_to_remove = pd.DataFrame(classification_cnt.loc[lambda x : x == 1])
cls_to_remove_list = cls_to_remove.index.tolist()

In [51]:
# Choose a cutoff value and create a list of classifications to be replaced

class_to_replace = list(classification_cnt[classification_cnt < 500 ].index)

# Replace in dataframe
for cls in class_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [52]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df,
                                        columns = ["APPLICATION_TYPE", 
                                                   "AFFILIATION", 
                                                   "CLASSIFICATION", 
                                                   "USE_CASE", 
                                                   "ORGANIZATION", 
                                                   "INCOME_AMT", 
                                                   "SPECIAL_CONSIDERATIONS"], 
                                        drop_first = True)


In [53]:
application_df_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
4,1,142590,1,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0


In [56]:
# Split our preprocessed data into our features and target arrays
y = application_df_dummies.IS_SUCCESSFUL
X = application_df_dummies.drop(columns="IS_SUCCESSFUL")


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [57]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [58]:
input_feature_count = len(X_train_scaled[0])
input_feature_count

37

In [60]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

input_feature_count = len(X_train_scaled[0])
input_feature_count

hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 10
hidden_nodes_layer4 = 3

nn = tf.keras.models.Sequential()

In [61]:

# First hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="sigmoid", input_dim = input_feature_count))

# Second hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Added a third hidden layer to improve accuracy
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="softmax"))

# Added a fourth hidden layer to improve accuracy
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 150)               5700      
                                                                 
 dense_11 (Dense)            (None, 50)                7550      
                                                                 
 dense_12 (Dense)            (None, 10)                510       
                                                                 
 dense_13 (Dense)            (None, 10)                110       
                                                                 
 dense_14 (Dense)            (None, 1)                 11        
                                                                 
Total params: 13,881
Trainable params: 13,881
Non-trainable params: 0
_________________________________________________________________


In [62]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [63]:
# Train the model

fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5963 - accuracy: 0.7036
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5672 - accuracy: 0.7266
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5617 - accuracy: 0.7269
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5596 - accuracy: 0.7296
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5595 - accuracy: 0.7292
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5571 - accuracy: 0.7297
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5565 - accuracy: 0.7284
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5552 - accuracy: 0.7302
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5541 - accuracy: 0.7292
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5535 - accura

In [64]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5540 - accuracy: 0.7278 - 522ms/epoch - 2ms/step
Loss: 0.5539519190788269, Accuracy: 0.7278134226799011


In [65]:
len(nn.get_weights())

10

In [66]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")